In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import cv2 # pip install opencv-python
from threading import Thread
import webview # pip install pywebview
import dash_bootstrap_components as dbc #pip install dash-bootstrap-components
from flask import Flask, Response # pip install flask

# Initialisation de l'application Dash
server = Flask(__name__)
app = dash.Dash(__name__, server=server, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Fonction pour capturer la vidéo en temps réel
def gen_frames():
    cap = cv2.VideoCapture(0)
    while True:
        success, frame = cap.read()
        if not success:
            break
        else:
            ret, buffer = cv2.imencode('.jpg', frame)
            frame = buffer.tobytes()
            yield (b'--frame\r\n'
                   b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')
    cap.release()


# Route pour la vidéo en temps réel
@server.route('/video_feed')
def video_feed():
    return Response(gen_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

# Layout de l'application Dash
app.layout = dbc.Container([
    dbc.Row([
        dbc.Col(html.H1("Camera Streaming"), width=12)
    ]),
    dbc.Row([
        dbc.Col(html.Img(src="/video_feed", id="video"), width=12)
    ])
])

if __name__ == '__main__':
    def run_app():
        app.run_server(host='localhost', port=8050,debug=False) # Lauch server http://localhost:8050/
    # run_app() # Permet de lancer l'application uniquement en mode web (Avec Navigateur)
    # Partie Webview - Application en mode natif - Windows (Sans Navigateur)
    # Conserve l'application Dash lancée # Utile pour Webview
    t = Thread(target=run_app)
    t.daemon = True
    t.start()
    # Creation de l'interface Webview incorporant la DASH APP
    window = webview.create_window('Photo Application', 'http://localhost:8050/', resizable=True, fullscreen=False)
    webview.start(debug=False)


##### To make an .exe #####
# I used autopytoexe.exe after install pyinstaller in my .venv
# I build an onefile .exe
# I didn't find how to use style.css in an external file (In assets...)
# I had to integrate style in my dash components in the code but it works

# I used autopytoexe.exe directly in .venv/Scripts/autopytoexe.exe
# I also got an error with some modules like dash_ag_grid when i opened my .exe after the first compile
# I got an error with dash_ag_grid, Dash_ag_grid has no hooks.py but it's possible to build without that.
# You just should add in "collect_data" in Advanced Module in autopytoexe.exe=> "dash_ag_grid"

"""
The error was
Traceback (most recent call last):
  File "main.py", line 7, in <module>
  File "<frozen importlib._bootstrap>", line 1360, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1331, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 935, in _load_unlocked
  File "PyInstaller\loader\pyimod02_importers.py", line 419, in exec_module
  File "dash_ag_grid\__init__.py", line 21, in <module>
FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Enzo\\AppData\\Local\\Temp\\_MEI212162\\dash_ag_grid\\package-info.json'
"""